# Clustering Crypto

In [303]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [304]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col = 0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [305]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[crypto_df['IsTrading']==True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [306]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df = crypto_df.dropna(axis=0, subset=['Algorithm'])
crypto_df.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000


In [307]:
# Remove the "IsTrading" column. 
crypto_df.drop(['IsTrading'], axis=1, inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [308]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [309]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [310]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_names = crypto_df[['CoinName']]
coin_names.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [311]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(['CoinName'], axis=1, inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [312]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [313]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [314]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.35120734,  0.96941878, -0.52267491],
       [-0.33449238,  0.96977443, -0.52309529],
       [ 2.30856427,  1.68091692, -0.69996008],
       ...,
       [ 0.33791595, -2.23279214,  0.36282444],
       [-0.18330188, -2.22993944,  0.43847127],
       [-0.28463453,  0.89074521, -0.2476224 ]])

In [315]:
index_values = (X.index.tolist())

In [316]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    X_pca, columns=['PC 1', 'PC 2', 'PC 3'], index=index_values)
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.351207,0.969419,-0.522675
404,-0.334492,0.969774,-0.523095
1337,2.308564,1.680917,-0.699960
BTC,-0.137076,-1.325500,0.164227
ETH,-0.135514,-2.025334,0.375027


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [317]:
# Create an elbow curve to find the best value for K.
inertia =[]
k = list(range(1,11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

C:\Users\Ryan\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [318]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
    
# Fit the model
model.fit(pcs_df)
    
# Predict Clusters
predictions = model.predict(pcs_df)
print(predictions)
    
# Create return DataFrame with predicted clusters
pcs_df['class'] = model.labels_

[0 0 0 3 3 3 0 3 3 3 0 3 0 0 3 0 3 3 0 0 3 3 3 3 3 0 3 3 3 0 3 0 3 3 0 0 3
 3 3 3 3 3 0 0 3 3 3 3 3 0 0 3 0 3 3 3 3 0 3 3 0 3 0 0 0 3 3 3 0 0 0 0 0 3
 3 3 0 0 3 0 3 0 0 3 3 3 3 0 0 3 0 3 3 0 0 3 0 0 3 3 0 0 3 0 0 3 0 3 0 3 0
 3 0 0 3 3 0 3 3 3 0 3 3 3 3 3 0 0 3 3 3 0 3 0 3 3 0 3 0 3 0 0 3 3 0 3 3 0
 0 3 0 3 0 0 0 3 3 3 3 0 0 0 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 3 0 3 0 0 3 0 3
 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 0 0 0 3 3 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 0
 0 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 3 0 3 0 0 3 0 3 3 0 3 3 0 3 0 0 0 3 0 0 3
 0 0 0 0 0 0 0 3 0 3 0 0 0 0 3 0 3 0 3 3 3 3 0 3 0 0 3 0 3 3 3 0 3 0 3 3 3
 0 3 0 3 0 0 0 3 0 3 3 3 3 3 0 0 3 0 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 3 0 0
 0 3 3 3 3 0 0 0 0 3 0 3 3 3 0 0 3 3 0 0 3 0 3 3 3 0 3 3 0 0 0 3 3 3 0 0 0
 3 3 0 3 3 3 3 0 1 1 3 3 3 0 1 0 0 0 0 3 3 3 3 0 0 0 3 0 3 0 0 0 0 3 0 0 3
 0 0 3 3 0 3 0 3 3 3 3 0 0 3 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 3 0 3 3 3 3
 0 0 0 0 3 0 0 3 0 0 3 1 3 0 3 3 0 0 3 0 3 3 3 3 3 0 3 0 3 0 0 3 0 0 0 0 0
 3 3 3 0 0 0 3 0 3 0 3 0 

In [319]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = crypto_df.join(pcs_df, how='inner')
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.351207,0.969419,-0.522675,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.334492,0.969774,-0.523095,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.308564,1.680917,-0.699960,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.137076,-1.325500,0.164227,3
ETH,Ethash,PoW,1.076842e+08,0,-0.135514,-2.025334,0.375027,3
...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,2.431610,0.606728,0.119358,0
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.349245,0.969338,-0.522716,0
BDX,CryptoNight,PoW,9.802226e+08,1400222610,0.337916,-2.232792,0.362824,3
ZEN,Equihash,PoW,7.296538e+06,21000000,-0.183302,-2.229939,0.438471,3


In [320]:
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['class'] = model.labels_
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.351207,0.969419,-0.522675,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.334492,0.969774,-0.523095,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.308564,1.680917,-0.699960,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.137076,-1.325500,0.164227,3
ETH,Ethash,PoW,1.076842e+08,0,-0.135514,-2.025334,0.375027,3
...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,2.431610,0.606728,0.119358,0
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.349245,0.969338,-0.522716,0
BDX,CryptoNight,PoW,9.802226e+08,1400222610,0.337916,-2.232792,0.362824,3
ZEN,Equihash,PoW,7.296538e+06,21000000,-0.183302,-2.229939,0.438471,3


In [321]:
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. # YOUR CODE HERE
clustered_df = clustered_df.join(coin_names, how='inner')
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.351207,0.969419,-0.522675,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.334492,0.969774,-0.523095,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.308564,1.680917,-0.699960,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.137076,-1.325500,0.164227,3,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.135514,-2.025334,0.375027,3,Ethereum
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,2.431610,0.606728,0.119358,0,ZEPHYR
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.349245,0.969338,-0.522716,0,Gapcoin
BDX,CryptoNight,PoW,9.802226e+08,1400222610,0.337916,-2.232792,0.362824,3,Beldex
ZEN,Equihash,PoW,7.296538e+06,21000000,-0.183302,-2.229939,0.438471,3,Horizen


In [322]:
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.351207,0.969419,-0.522675,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.334492,0.969774,-0.523095,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.308564,1.680917,-0.699960,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.137076,-1.325500,0.164227,3,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.135514,-2.025334,0.375027,3,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.168216,-1.111074,0.024709,3,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.396133,1.169778,-0.484510,0,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.144593,-2.256026,0.422444,3,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.133950,-2.025406,0.374994,3,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.183301,-2.229939,0.438471,3,ZCash


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [323]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
    width=800,
    hover_name='CoinName',
    hover_data=['Algorithm', 'TotalCoinsMined', 'TotalCoinSupply']
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [324]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class'])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [325]:
# Print the total number of tradable cryptocurrencies.
count = clustered_df['CoinName'].count()
print(f'There are {count} tradable cryptocurrencies')

There are 532 tradable cryptocurrencies


In [326]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
cluster_df = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
clustered_scaled = MinMaxScaler().fit_transform(cluster_df)
clustered_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [327]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
index_values= (cluster_df.index.tolist())
plot_df = pd.DataFrame(data = clustered_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=index_values)
plot_df

,TotalCoinSupply,TotalCoinsMined
42,4.200000e-11,0.000000e+00
404,5.320000e-04,1.065855e-03
1337,3.141593e-01,2.957551e-02
BTC,2.100000e-05,1.810842e-05
ETH,0.000000e+00,1.087731e-04
...,...,...
ZEPH,2.000000e-03,2.020225e-03
GAP,2.500000e-04,1.508199e-05
BDX,1.400223e-03,9.901351e-04
ZEN,2.100000e-05,7.370282e-06


In [328]:
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df['CoinName']

In [329]:
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
class_df = clustered_df['class']
plot_df = plot_df.join(class_df, how="inner")

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,3
ETH,0.000000e+00,0.000109,Ethereum,3
LTC,8.400000e-05,0.000064,Litecoin,3
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,3
ETC,2.100000e-04,0.000115,Ethereum Classic,3
ZEC,2.100000e-05,0.000007,ZCash,3


In [330]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="class")


:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)